In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from itertools import combinations

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, Interaction
from pyspark.ml.regression import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline

In [2]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'regressionHW'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


In [3]:
''' get the data '''
# load the data
fil = '../../data/Concrete_Data.csv'
schem = StructType([StructField('cement', FloatType()), StructField('slag', FloatType()),
                    StructField('flyash', FloatType()), StructField('water', FloatType()),
                    StructField('superplasticizer', FloatType()), StructField('coarseaggregate', FloatType()),
                    StructField('fineaggregate', FloatType()), StructField('age', FloatType()),
                    StructField('csMPa', FloatType())])
concrete = spark.read.format('csv').options(header=True).schema(schem).load(fil)

# add an ID - don't actually care if it's monotonic
concrete = concrete.select(monotonically_increasing_id().alias('id'), '*')

# talk
cnt = concrete.count()
print('%d records'%cnt)
concrete.show(truncate=False)

1030 records
+---+------+-----+------+-----+----------------+---------------+-------------+-----+-----+
|id |cement|slag |flyash|water|superplasticizer|coarseaggregate|fineaggregate|age  |csMPa|
+---+------+-----+------+-----+----------------+---------------+-------------+-----+-----+
|0  |540.0 |0.0  |0.0   |162.0|2.5             |1040.0         |676.0        |28.0 |79.99|
|1  |540.0 |0.0  |0.0   |162.0|2.5             |1055.0         |676.0        |28.0 |61.89|
|2  |332.5 |142.5|0.0   |228.0|0.0             |932.0          |594.0        |270.0|40.27|
|3  |332.5 |142.5|0.0   |228.0|0.0             |932.0          |594.0        |365.0|41.05|
|4  |198.6 |132.4|0.0   |192.0|0.0             |978.4          |825.5        |360.0|44.3 |
|5  |266.0 |114.0|0.0   |228.0|0.0             |932.0          |670.0        |90.0 |47.03|
|6  |380.0 |95.0 |0.0   |228.0|0.0             |932.0          |594.0        |365.0|43.7 |
|7  |380.0 |95.0 |0.0   |228.0|0.0             |932.0          |594.0        

### Data Prep

In [4]:
''' handle missing values '''
# check for missing values
nullCounts = {colm:concrete.select(colm).where(col(colm).isNull()).count() for colm in concrete.columns}
nullCounts = {colm:(ncnt, ncnt/cnt) for (colm, ncnt) in nullCounts.items()}
nullCountsDF = pd.DataFrame(nullCounts).T.reset_index(drop=False).sort_values(1, ascending=False)
nullCountsDF.columns = ['Column', 'Freq.', 'Rel. Freq.']
nullCountsDF = nullCountsDF.merge(pd.DataFrame([[colm.name, colm.dataType] for colm in concrete.schema], columns=['Column', 'Type']),
                                how='inner', on=['Column'])

# talk
display(nullCountsDF)

# remove
#concrete = concrete.dropna(how='any')

# talk some more
print('%d records'%concrete.count())

,Column,Freq.,Rel. Freq.,Type
0,id,0.0,0.0,LongType
1,cement,0.0,0.0,FloatType
2,slag,0.0,0.0,FloatType
3,flyash,0.0,0.0,FloatType
4,water,0.0,0.0,FloatType
5,superplasticizer,0.0,0.0,FloatType
6,coarseaggregate,0.0,0.0,FloatType
7,fineaggregate,0.0,0.0,FloatType
8,age,0.0,0.0,FloatType
9,csMPa,0.0,0.0,FloatType


1030 records


In [5]:
# prepare the response
concrete = concrete.withColumnRenamed('csMPa', 'label')

In [6]:
''' create the squared features'''
# make the lists
features = [c for c in concrete.columns if c not in (['label', 'id'])]
# square them
for feat in features:
    concrete = concrete.withColumn(feat+'_sq', col(feat)*col(feat))
# talk
concrete.show(truncate=False)

+---+------+-----+------+-----+----------------+---------------+-------------+-----+-----+---------+---------+---------+--------+-------------------+------------------+----------------+--------+
|id |cement|slag |flyash|water|superplasticizer|coarseaggregate|fineaggregate|age  |label|cement_sq|slag_sq  |flyash_sq|water_sq|superplasticizer_sq|coarseaggregate_sq|fineaggregate_sq|age_sq  |
+---+------+-----+------+-----+----------------+---------------+-------------+-----+-----+---------+---------+---------+--------+-------------------+------------------+----------------+--------+
|0  |540.0 |0.0  |0.0   |162.0|2.5             |1040.0         |676.0        |28.0 |79.99|291600.0 |0.0      |0.0      |26244.0 |6.25               |1081600.0         |456976.0        |784.0   |
|1  |540.0 |0.0  |0.0   |162.0|2.5             |1055.0         |676.0        |28.0 |61.89|291600.0 |0.0      |0.0      |26244.0 |6.25               |1113025.0         |456976.0        |784.0   |
|2  |332.5 |142.5|0.0   |

In [7]:
''' create the interaction features '''
# iterate over all pairs
for (a, b) in combinations(features, 2):
    print('%s <-> %s'%(a, b))
    intr = Interaction(inputCols=[a,b], outputCol=a+'_'+b)
    concrete = intr.transform(concrete)
# talk
concrete.show(truncate=False)

cement <-> slag
cement <-> flyash
cement <-> water
cement <-> superplasticizer
cement <-> coarseaggregate
cement <-> fineaggregate
cement <-> age
slag <-> flyash
slag <-> water
slag <-> superplasticizer
slag <-> coarseaggregate
slag <-> fineaggregate
slag <-> age
flyash <-> water
flyash <-> superplasticizer
flyash <-> coarseaggregate
flyash <-> fineaggregate
flyash <-> age
water <-> superplasticizer
water <-> coarseaggregate
water <-> fineaggregate
water <-> age
superplasticizer <-> coarseaggregate
superplasticizer <-> fineaggregate
superplasticizer <-> age
coarseaggregate <-> fineaggregate
coarseaggregate <-> age
fineaggregate <-> age
+---+------+-----+------+-----+----------------+---------------+-------------+-----+-----+---------+---------+---------+--------+-------------------+------------------+----------------+--------+--------------------+-------------+-----------------+-----------------------+----------------------+--------------------+--------------------+-----------+--------

In [8]:
''' prepare the features '''
# get the features
features = [c for c in concrete.columns if c not in (['label', 'id'])]
print(features)

# create & scale the features vector
assr = VectorAssembler(inputCols=features, outputCol='features_raw')
scalr = MinMaxScaler(inputCol='features_raw', outputCol='features')
pipe = Pipeline(stages=[assr, scalr]).fit(concrete)
concrete = pipe.transform(concrete).drop('features_raw')

# talk
display(concrete.limit(10).toPandas())
concrete.select('id', 'features', 'label').show(truncate=False)
concrete.select('features').take(1)
print('First row features = %s'%concrete.select('features').take(1)[0])

['cement', 'slag', 'flyash', 'water', 'superplasticizer', 'coarseaggregate', 'fineaggregate', 'age', 'cement_sq', 'slag_sq', 'flyash_sq', 'water_sq', 'superplasticizer_sq', 'coarseaggregate_sq', 'fineaggregate_sq', 'age_sq', 'cement_slag', 'cement_flyash', 'cement_water', 'cement_superplasticizer', 'cement_coarseaggregate', 'cement_fineaggregate', 'cement_age', 'slag_flyash', 'slag_water', 'slag_superplasticizer', 'slag_coarseaggregate', 'slag_fineaggregate', 'slag_age', 'flyash_water', 'flyash_superplasticizer', 'flyash_coarseaggregate', 'flyash_fineaggregate', 'flyash_age', 'water_superplasticizer', 'water_coarseaggregate', 'water_fineaggregate', 'water_age', 'superplasticizer_coarseaggregate', 'superplasticizer_fineaggregate', 'superplasticizer_age', 'coarseaggregate_fineaggregate', 'coarseaggregate_age', 'fineaggregate_age']


,id,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,label,...,water_coarseaggregate,water_fineaggregate,water_age,superplasticizer_coarseaggregate,superplasticizer_fineaggregate,superplasticizer_age,coarseaggregate_fineaggregate,coarseaggregate_age,fineaggregate_age,features
0,0,540.000000,0.000000,0.0,162.0,2.5,1040.000000,676.0,28.0,79.989998,...,[168480.0],[109512.0],[4536.0],[2600.0],[1690.0],[70.0],[703040.0],[29120.0],[18928.0],"(1.0, 0.0, 0.0, 0.3210862454322655, 0.07763974..."
1,1,540.000000,0.000000,0.0,162.0,2.5,1055.000000,676.0,28.0,61.889999,...,[170910.0],[109512.0],[4536.0],[2637.5],[1690.0],[70.0],[713180.0],[29540.0],[18928.0],"(1.0, 0.0, 0.0, 0.3210862454322655, 0.07763974..."
2,2,332.500000,142.500000,0.0,228.0,0.0,932.000000,594.0,270.0,40.270000,...,[212496.0],[135432.0],[61560.0],[0.0],[0.0],[0.0],[553608.0],[251640.0],[160380.0],"(0.526255707762557, 0.39649416366168144, 0.0, ..."
3,3,332.500000,142.500000,0.0,228.0,0.0,932.000000,594.0,365.0,41.049999,...,[212496.0],[135432.0],[83220.0],[0.0],[0.0],[0.0],[553608.0],[340180.0],[216810.0],"(0.526255707762557, 0.39649416366168144, 0.0, ..."
4,4,198.600006,132.399994,0.0,192.0,0.0,978.400024,825.5,360.0,44.299999,...,[187852.8046875],[158496.0],[69120.0],[0.0],[0.0],[0.0],[807669.2201538086],[352224.0087890625],[297180.0],"(0.22054795914044661, 0.3683917533249004, 0.0,..."
5,5,266.000000,114.000000,0.0,228.0,0.0,932.000000,670.0,90.0,47.029999,...,[212496.0],[152760.0],[20520.0],[0.0],[0.0],[0.0],[624440.0],[83880.0],[60300.0],"(0.3744292237442922, 0.31719533092934515, 0.0,..."
6,6,380.000000,95.000000,0.0,228.0,0.0,932.000000,594.0,365.0,43.700001,...,[212496.0],[135432.0],[83220.0],[0.0],[0.0],[0.0],[553608.0],[340180.0],[216810.0],"(0.634703196347032, 0.264329442441121, 0.0, 0...."
7,7,380.000000,95.000000,0.0,228.0,0.0,932.000000,594.0,28.0,36.450001,...,[212496.0],[135432.0],[6384.0],[0.0],[0.0],[0.0],[553608.0],[26096.0],[16632.0],"(0.634703196347032, 0.264329442441121, 0.0, 0...."
8,8,266.000000,114.000000,0.0,228.0,0.0,932.000000,670.0,28.0,45.849998,...,[212496.0],[152760.0],[6384.0],[0.0],[0.0],[0.0],[624440.0],[26096.0],[18760.0],"(0.3744292237442922, 0.31719533092934515, 0.0,..."
9,9,475.000000,0.000000,0.0,228.0,0.0,932.000000,594.0,28.0,39.290001,...,[212496.0],[135432.0],[6384.0],[0.0],[0.0],[0.0],[553608.0],[26096.0],[16632.0],"(0.8515981735159817, 0.0, 0.0, 0.8482428078025..."


+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|id |features                                                                                                                                                                                                                                                                                                                                                                            

In [9]:
# check for multicollinearity
# high: 
corr = Correlation.corr(concrete, column='features', method='pearson')
corrdf = pd.DataFrame(index=features, data=corr.collect()[0][0].toArray(), columns=features)
display(corrdf)

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,cement_sq,slag_sq,...,water_superplasticizer,water_coarseaggregate,water_fineaggregate,water_age,superplasticizer_coarseaggregate,superplasticizer_fineaggregate,superplasticizer_age,coarseaggregate_fineaggregate,coarseaggregate_age,fineaggregate_age
cement,1.000000,-0.275216,-0.397467,-0.081587,0.092386,-0.109349,-0.222718,0.081946,0.982918,-0.263361,...,0.046479,-0.122675,-0.303795,0.081596,0.056413,0.102353,0.052432,-0.289801,0.079233,0.051942
slag,-0.275216,1.000000,-0.323580,0.107252,0.043270,-0.283999,-0.281603,-0.044246,-0.252955,0.938881,...,0.055139,-0.077578,-0.148709,-0.029918,0.022581,0.013891,0.011282,-0.439001,-0.059093,-0.069224
flyash,-0.397467,-0.323580,1.000000,-0.256984,0.377503,-0.009961,0.079109,-0.154371,-0.377955,-0.321191,...,0.435933,-0.265882,-0.178359,-0.174483,0.409375,0.347125,0.232739,0.080499,-0.149140,-0.141664
water,-0.081587,0.107252,-0.256984,1.000000,-0.657533,-0.182294,-0.450661,0.277618,-0.084376,0.123723,...,-0.580579,0.791801,0.580572,0.353335,-0.684416,-0.674862,-0.461811,-0.499785,0.260321,0.221800
superplasticizer,0.092386,0.043270,0.377503,-0.657533,1.000000,-0.265999,0.222691,-0.192700,0.094161,-0.032298,...,0.985844,-0.760687,-0.473556,-0.234908,0.992174,0.989603,0.648698,0.003560,-0.200512,-0.170206
coarseaggregate,-0.109349,-0.283999,-0.009961,-0.182294,-0.265999,1.000000,-0.178481,-0.003016,-0.111007,-0.199890,...,-0.302954,0.452401,-0.311224,-0.024739,-0.176358,-0.285178,-0.118615,0.516810,0.044597,-0.008118
fineaggregate,-0.222718,-0.281603,0.079109,-0.450661,0.222691,-0.178481,1.000000,-0.156095,-0.240355,-0.295108,...,0.178529,-0.508146,0.456093,-0.202127,0.202625,0.307266,0.180348,0.746981,-0.159743,-0.053369
age,0.081946,-0.044246,-0.154371,0.277618,-0.192700,-0.003016,-0.156095,1.000000,0.072464,-0.052414,...,-0.209001,0.248610,0.105381,0.991072,-0.193188,-0.182441,0.109230,-0.133359,0.997099,0.985084
cement_sq,0.982918,-0.252955,-0.377955,-0.084376,0.094161,-0.111007,-0.240355,0.072464,1.000000,-0.245626,...,0.048750,-0.126132,-0.321706,0.070980,0.056047,0.104170,0.051071,-0.310421,0.071478,0.039727
slag_sq,-0.263361,0.938881,-0.321191,0.123723,-0.032298,-0.199890,-0.295108,-0.052414,-0.245626,1.000000,...,-0.020682,-0.009810,-0.139581,-0.039160,-0.047081,-0.058338,-0.036722,-0.393011,-0.062502,-0.074709


## Modeling

In [10]:
# split for cross-val
trainPerc = 0.7
randSeed = 42
tran, test = concrete.select('id', 'label', 'features').randomSplit([trainPerc, 1.0 - trainPerc], seed=randSeed)

# talk
print('Training Cases')
tran.select('id').show()
print('Testing Cases')
test.select('id').show()

Training Cases
+---+
| id|
+---+
|  0|
|  1|
|  3|
|  4|
|  5|
|  7|
| 10|
| 11|
| 12|
| 16|
| 17|
| 18|
| 20|
| 22|
| 25|
| 26|
| 27|
| 31|
| 33|
| 36|
+---+
only showing top 20 rows

Testing Cases
+---+
| id|
+---+
|  2|
|  6|
|  8|
|  9|
| 13|
| 14|
| 15|
| 19|
| 21|
| 23|
| 24|
| 28|
| 29|
| 30|
| 32|
| 34|
| 35|
| 39|
| 42|
| 43|
+---+
only showing top 20 rows



In [11]:
''' set up the estimators & param grids '''
models = {}

# linear regression
linreg = LinearRegression()
params = (ParamGridBuilder().addGrid(linreg.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]).build())
paramNames = ['elasticnetparam']
models['linear regression'] = [linreg, params, paramNames, None, None]

# random forest
ranfor = RandomForestRegressor(numTrees=20)
params = (ParamGridBuilder().addGrid(ranfor.maxBins, [20, 40, 80, 100])\
              .addGrid(ranfor.maxDepth, [5, 10, 30]).build())
paramNames = ['maxbins', 'maxdepth']
models['random forest'] = [ranfor, params, paramNames, None, None]

# gradient boosting trees
gradbst = GBTRegressor(maxIter=20)
params = (ParamGridBuilder().addGrid(gradbst.maxBins, [20, 40, 80, 100])\
              .addGrid(gradbst.maxDepth, [5, 10, 30]).build())
paramNames = ['maxbins', 'maxdepth']
models['gradient boost'] = [gradbst, params, paramNames, None, None]

In [ ]:
''' run the models '''
# number of cv folds
folds = 5
# define the evaulation function
evl = RegressionEvaluator(metricName='rmse')

# iterate over models
for (model, stuff) in models.items():
    print('Cross Validator: %s'%model)
    # execute
    cv = CrossValidator(estimator=stuff[0], estimatorParamMaps=stuff[1], evaluator=evl, numFolds=folds)
    fitModel = cv.fit(concrete.select('features', 'label'))
    # get the best
    bestModel = fitModel.bestModel
    # evaluate performance on the test set
    testRMSE = evl.evaluate(bestModel.transform(test.select('features', 'label')))
    print('\tBest Model Test RMSE = %0.3f'%testRMSE)    
    # get best parameters
    bestParams = bestModel.extractParamMap()
    for (key, val) in bestParams.items():
        for parm in stuff[2]:
            if parm in key.name.lower():
                print('\t%s = %0.2f'%(key, val))
                break
    # save stuff
    models[model][3] = fitModel
    models[model][4] = testRMSE

Cross Validator: linear regression


In [ ]:
# look at linear regression coefficients
bm = models['linear regression'][3].bestModel
summ = bm.summary
summ.predictions.describe().withColumn('Diff', col('prediction') - col('label')).show(truncate=False)
print('Best Model RMSE = %0.3f'%summ.rootMeanSquaredError)

# get a nice model coefficients table
coefs = pd.concat([pd.DataFrame(index=['Intercept'], data=[bm.intercept], columns=['Coefficient']),
                   pd.DataFrame(index=features, data=bm.coefficients.toArray(), columns=['Coefficient'])])
#coefs['Std. Error'] = bm.summary.coefficientStandardErrors
#coefs['pValue'] = bm.summary.pValues
# make an absolute coef column temporarily for sorting
coefs['tmp'] = coefs['Coefficient'].abs()
coefs.loc['Intercept', 'tmp'] = np.inf
coefs = coefs.sort_values(by='tmp', ascending=False).drop(columns='tmp')

# talk
display(coefs)

In [ ]:
# view feature importances for random forest
imports = models['random forest'][3].bestModel.featureImportances.toArray()
imports = pd.DataFrame(index=features, data=imports, columns=['Importance']).sort_values(by='Importance', ascending=False, inplace=False)
display(imports)

In [ ]:
# view feature importances for gradient boost
imports = models['gradient boost'][3].bestModel.featureImportances.toArray()
imports = pd.DataFrame(index=features, data=imports, columns=['Importance']).sort_values(by='Importance', ascending=False, inplace=False)
display(imports)

In [ ]:
sc.stop()